In [1]:
import numpy as np
import pandas as pd
import os
import glob
import nltk

In [2]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from itertools import chain

In [3]:
tokenizer = RegexpTokenizer("\w+")
stemmer = nltk.PorterStemmer()

In [4]:
import copy

In [110]:
%%time

loc = os.getcwd()
term_freq = {}
class_freq = {}
file_class = {}
doc_Id = {}

empty_term_freq = {}
for i in range(1000):
    empty_term_freq[i] = 0

empty_class_freq = {}
empty_class_freq["business"] = 0
empty_class_freq["entertainment"] = 0
empty_class_freq["politics"] = 0
empty_class_freq["sport"] = 0
empty_class_freq["tech"] = 0
    
for i,file in enumerate(glob.glob(loc + "/bbc-fulltext/*/*/*.txt")):
    document = open(file).read()
    doc_Id[i] = file
    file_class[i] = file.split("/")[-2]
    tokenized = tokenizer.tokenize(document)
    for word in tokenized:
        stemmed = stemmer.stem(word)
        if stemmed not in term_freq.keys():
            term_freq[stemmed] = copy.deepcopy(empty_term_freq)
        term_freq[stemmed][i] += 1
    
    

CPU times: user 21.2 s, sys: 157 ms, total: 21.3 s
Wall time: 21.3 s


In [6]:
%%time
#remove stop words from dict
stop_word = set(stopwords.words("english"))
for word in stop_word:
    stemmed = stemmer.stem(word)
    if stemmed in term_freq.keys():
        del term_freq[stemmed]

CPU times: user 13.8 ms, sys: 0 ns, total: 13.8 ms
Wall time: 137 ms


In [7]:
tf = pd.DataFrame(term_freq)

In [8]:
#tf.to_csv("VSMTermFrequency.csv")

In [9]:
#cf.to_csv("VSMClassFrequency.csv")

In [12]:
#class_ = pd.DataFrame(file_class)

In [10]:
#x = pd.concat([tf, class_], axis = 1)

NameError: name 'class_' is not defined

In [13]:
doc_freq = (tf > 0).sum(axis = 0)
inv_doc_freq = np.log10(1000/doc_freq)

In [14]:
%%time
tf = 1 + np.log10(tf)
tf.replace(to_replace = -np.inf, value = 0, inplace = True)

/home/vedant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


CPU times: user 3.68 s, sys: 126 ms, total: 3.81 s
Wall time: 3.71 s


In [15]:
tf_idf = tf.multiply(inv_doc_freq, axis = 1)

In [16]:
business_doc_freq = (tf.drop(index=range(200,1000)) > 0).sum(axis = 0)
business_tf_idf_cf = (tf_idf.drop(index = (range(200,1000)))).multiply(business_doc_freq)

In [17]:
politics_doc_freq = (tf.drop(index=chain(range(200), range(400,1000))) > 0).sum(axis = 0)
politics_tf_idf_cf = (tf_idf.drop(index = chain(range(200), range(400,1000)))).multiply(politics_doc_freq)

In [18]:
entertainment_doc_freq = (tf.drop(index=chain(range(400), range(600,1000))) > 0).sum(axis = 0)
entertainment_tf_idf_cf = (tf_idf.drop(index = chain(range(400), range(600,1000)))).multiply(entertainment_doc_freq)

In [19]:
sport_doc_freq = (tf.drop(index=chain(range(600), range(800,1000))) > 0).sum(axis = 0)
sport_tf_idf_cf = (tf_idf.drop(index=chain(range(600), range(800,1000)))).multiply(sport_doc_freq)

In [20]:
tech_doc_freq = (tf.drop(index = range(800)) > 0).sum(axis = 0)
tech_tf_idf_cf = (tf_idf.drop(index = range(800))).multiply(tech_doc_freq)

In [32]:
tf_idf_cf = business_tf_idf_cf.append([politics_tf_idf_cf, entertainment_tf_idf_cf,sport_tf_idf_cf, tech_tf_idf_cf])

In [54]:
norm_tf_idf_cf = tf_idf_cf.multiply( 1/ (((tf_idf_cf**2).sum(axis = 1))**0.5), axis= 0)

In [58]:
dict_word_index = {}
for i,word in enumerate(norm_tf_idf_cf.columns):
    dict_word_index[word] = i
    

In [65]:
empty_query_vector = np.zeros(14270)


In [64]:
query = input("Search")

SearchGolden boost


In [76]:
tokenized = tokenizer.tokenize(query)
query_vector = copy.deepcopy(empty_query_vector)
for word in tokenized:
    stemmed = stemmer.stem(word)
    print(stemmed)
    print(dict_word_index[stemmed])
    query_vector[dict_word_index[stemmed]] = 1
    print(query_vector[dict_word_index[stemmed]])
    

golden
0
1.0
boost
2
1.0


In [100]:
x = {}
for i,row in enumerate(norm_tf_idf_cf.iterrows()):
    x[i] = np.matmul(row[1], query_vector.T)[1]

In [128]:
for i,doc in enumerate(x):
    file = doc_Id[doc[0]]
    document = open(file).read()
    print(doc, end = "\t")
    print(document)

    if i == 9:
        break

(41, 0.24418541303084357)	Salary scandal in Cameroon

Cameroon says widespread corruption in its finance ministry has cost it 1bn CFA francs ($2m; £1m) a month.

About 500 officials are accused of either awarding themselves extra money or claiming salaries for "non-existent" workers. Prime Minister Ephraim Inoni, who vowed to tackle corruption when he came to office last year, said those found guilty would face tough punishments. The scam is believed to have begun in 1994.

The prime minister's office said the alleged fraud was uncovered during an investigation into the payroll at the ministry. In certain cases, staff are said to have lied about their rank and delayed their retirement in order to boost their earnings. The prime minister's office said auditors had found "irregularities in the career structure of certain civil servants". It added that the staff in question "appear to have received unearned salaries, boosting the payroll".

Fidelis Nanga, a journalist based in the Cameroo

In [107]:
x = sorted(x.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)

[(41, 0.24418541303084357),
 (0, 0.22791223522868218),
 (161, 0.19951875403120228),
 (191, 0.19503263727775166),
 (35, 0.19117338941734166),
 (140, 0.18352290179512679),
 (118, 0.17960897255102673),
 (192, 0.1720072113126464),
 (44, 0.1719313892427333),
 (186, 0.17117664447849254),
 (158, 0.1665633259005625),
 (119, 0.163209170614346),
 (29, 0.16218007777083546),
 (424, 0.1573516527118431),
 (84, 0.1534093054497252),
 (550, 0.1521764078827745),
 (169, 0.14346228261664257),
 (135, 0.14016780904664336),
 (102, 0.13792854538562507),
 (17, 0.13786825240278042),
 (38, 0.13717732066040353),
 (23, 0.13462796907287514),
 (452, 0.13391073047088348),
 (78, 0.1336207420520294),
 (576, 0.1329737342417706),
 (61, 0.13159418210258575),
 (28, 0.13061458074428556),
 (95, 0.13006421796449696),
 (165, 0.12953052966950648),
 (153, 0.126736036168943),
 (569, 0.12603912247683993),
 (7, 0.12418129719217245),
 (407, 0.12253787680441543),
 (1, 0.12171897425244893),
 (112, 0.12160542051479395),
 (156, 0.117980